# Gensim의 Doc2Vec

[Gensim 홈페이지 참고](https://radimrehurek.com/gensim/auto_examples/tutorials/run_doc2vec_lee.html#)

In [ ]:
!pip install --upgrade gensim

**훈련, 테스트 데이터 준비**

In [2]:
import os
import gensim

In [4]:
# 훈련, 테스트 데이터 설정
test_data_dir = os.path.join(gensim.__path__[0], 'test', 'test_data')
lee_train_file = os.path.join(test_data_dir, 'lee_background.cor')
lee_test_file = os.path.join(test_data_dir, 'lee.cor')

**텍스트 전처리**

In [5]:
import smart_open

def read_corpus(fname, tokens_only=False):
    with smart_open.open(fname, encoding="iso-8859-1") as f:
        for i, line in enumerate(f):
            tokens = gensim.utils.simple_preprocess(line)
            if tokens_only:
                yield tokens
            else:
                # 학습 데이터의 경우 태그 추가
                yield gensim.models.doc2vec.TaggedDocument(tokens, [i])

train_corpus = list(read_corpus(lee_train_file))
test_corpus = list(read_corpus(lee_test_file, tokens_only=True))

In [6]:
print(train_corpus[:2])

[TaggedDocument(words=['hundreds', 'of', 'people', 'have', 'been', 'forced', 'to', 'vacate', 'their', 'homes', 'in', 'the', 'southern', 'highlands', 'of', 'new', 'south', 'wales', 'as', 'strong', 'winds', 'today', 'pushed', 'huge', 'bushfire', 'towards', 'the', 'town', 'of', 'hill', 'top', 'new', 'blaze', 'near', 'goulburn', 'south', 'west', 'of', 'sydney', 'has', 'forced', 'the', 'closure', 'of', 'the', 'hume', 'highway', 'at', 'about', 'pm', 'aedt', 'marked', 'deterioration', 'in', 'the', 'weather', 'as', 'storm', 'cell', 'moved', 'east', 'across', 'the', 'blue', 'mountains', 'forced', 'authorities', 'to', 'make', 'decision', 'to', 'evacuate', 'people', 'from', 'homes', 'in', 'outlying', 'streets', 'at', 'hill', 'top', 'in', 'the', 'new', 'south', 'wales', 'southern', 'highlands', 'an', 'estimated', 'residents', 'have', 'left', 'their', 'homes', 'for', 'nearby', 'mittagong', 'the', 'new', 'south', 'wales', 'rural', 'fire', 'service', 'says', 'the', 'weather', 'conditions', 'which', '

In [7]:
print(test_corpus[:2])

[['the', 'national', 'executive', 'of', 'the', 'strife', 'torn', 'democrats', 'last', 'night', 'appointed', 'little', 'known', 'west', 'australian', 'senator', 'brian', 'greig', 'as', 'interim', 'leader', 'shock', 'move', 'likely', 'to', 'provoke', 'further', 'conflict', 'between', 'the', 'party', 'senators', 'and', 'its', 'organisation', 'in', 'move', 'to', 'reassert', 'control', 'over', 'the', 'party', 'seven', 'senators', 'the', 'national', 'executive', 'last', 'night', 'rejected', 'aden', 'ridgeway', 'bid', 'to', 'become', 'interim', 'leader', 'in', 'favour', 'of', 'senator', 'greig', 'supporter', 'of', 'deposed', 'leader', 'natasha', 'stott', 'despoja', 'and', 'an', 'outspoken', 'gay', 'rights', 'activist'], ['cash', 'strapped', 'financial', 'services', 'group', 'amp', 'has', 'shelved', 'million', 'plan', 'to', 'buy', 'shares', 'back', 'from', 'investors', 'and', 'will', 'raise', 'million', 'in', 'fresh', 'capital', 'after', 'profits', 'crashed', 'in', 'the', 'six', 'months', 'to'

**모델 학습**

In [8]:
model = gensim.models.doc2vec.Doc2Vec(vector_size=50, min_count=2, epochs=40)

In [9]:
# 문서에서 어휘를 구축
model.build_vocab(train_corpus)

In [10]:
# 'penalty'가 훈련 데이터에 나타난 횟수를 확인
print(f"Word 'penalty' appeared {model.wv.get_vecattr('penalty', 'count')} times in the training corpus.")

Word 'penalty' appeared 4 times in the training corpus.


In [11]:
# 모델 학습
model.train(train_corpus, total_examples=model.corpus_count, epochs=model.epochs)

In [12]:
# 주어진 문서에 대한 벡터를 추론
vector = model.infer_vector(['only', 'you', 'can', 'prevent', 'forest', 'fires'])
print(vector)

[-0.23025462 -0.24345836 -0.05305537  0.33056828 -0.05839749  0.06090755
  0.07358902  0.02671815 -0.23019657 -0.09091736  0.21842927 -0.10298009
 -0.03166628 -0.04625431 -0.10176978 -0.20463231  0.07202011  0.14221962
  0.20162293 -0.1059368   0.07823508  0.03877781  0.2416942  -0.0070712
 -0.05378868  0.0255782  -0.18319151  0.04988231 -0.19557436 -0.00587856
  0.422276   -0.01141432  0.20319301  0.12072812  0.21385723  0.11297145
  0.03970773 -0.24217291 -0.11216845 -0.02282613 -0.05550453  0.09344329
  0.00782073 -0.08197795  0.23821136  0.04133388 -0.08900682 -0.10254689
  0.10326959  0.02780459]


**모델 테스트**

In [14]:
import random

# 테스트 데이터에서 임의의 문서를 선택하고 모델에서 벡터를 추론
doc_id = random.randint(0, len(test_corpus) - 1)
inferred_vector = model.infer_vector(test_corpus[doc_id])
sims = model.dv.most_similar([inferred_vector], topn=len(model.dv))

# 임의의 문서에 대해 훈련 데이터와 비교하여 가장 유사한/중간/가장 유사하지 않은 문서 확인
print('Test Document ({}): «{}»\n'.format(doc_id, ' '.join(test_corpus[doc_id])))
print(u'SIMILAR/DISSIMILAR DOCS PER MODEL %s:\n' % model)

for label, index in [('MOST', 0), ('MEDIAN', len(sims)//2), ('LEAST', len(sims) - 1)]:
    print(u'%s %s: «%s»\n' % (label, sims[index], ' '.join(train_corpus[sims[index][0]].words)))

Test Document (42): «pope john paul ii urged delegates at major summit on sustainable growth on sunday to pursue development that protects the environment and social justice in comments to tourists and the faithful at his summer residence southeast of rome the pope said god had put humans on earth to be his administrators of the land to cultivate it and take care of it in world ever more interdependent peace justice and the safekeeping of creation cannot but be the fruit of joint commitment of all in pursuing the common good john paul said»

SIMILAR/DISSIMILAR DOCS PER MODEL Doc2Vec<dm/m,d50,n5,w5,mc2,s0.001,t3>:

MOST (186, 0.6500346064567566): «united nationals secretary general kofi annan has accepted the nobel peace prize in the norwegian capital oslo declaring that to save one life is to save humanity itself mr annan told gala audience the world must respect the individual whose fundamental rights he says have been sacrificed too often for the good of the state the year old un chi